In [4]:
import pandas as pd

In [9]:
data = pd.read_csv('data/train_preprocessed.csv')

In [12]:
from environment import Environment

In [13]:
env = Environment(data)

KeyError: 'day'

In [ ]:
model = None # load 